# Sugarcane disease classification

This Repository consists of using Transfer Learning Method to detect types of Disease Sugarcane Plant can have.
Disease have following categories:
1. grassy shoot
2. leaf spot
3. nitrogen
4. orange rust
5. pyrilla
6. redrot
7. smut
8. wholly aphid
9. wilt
10. yellow leaf disease
11. leaf scald

In [27]:
import numpy as np
import os
import shutil
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
from keras.applications.vgg16 import VGG16
from keras.applications import xception
from keras.applications import inception_v3
from keras.applications.vgg16 import preprocess_input
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, accuracy_score

In [28]:
INPUT_SIZE=224
POOLING='avg'

#### Read Images, and Load them as Numpy array

In [29]:
def extract_features(path):
    directory_lists=os.listdir(path)
    X=[]
    Y=[]
    count=0
    if ('.DS_Store' in directory_lists):
            directory_lists.remove('.DS_Store')
    for d in directory_lists:
        nest=os.listdir(path+"/"+d)
        if ('.DS_Store' in nest):
            nest.remove('.DS_Store')
        for f in nest:
            img = image.load_img(path+"/"+d+"/"+f, target_size=(224, 224))
            img_data = image.img_to_array(img)
            img_data = preprocess_input(img_data)
            X.append(img_data)
            Y.append(count)
        count+=1
    X=np.array(X)
    y=np.array(Y)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

    return X_train, X_test, y_train, y_test 

#### We are using Transfer Learning using features extracted from VGG16, and Xception model trained on Image Net, we are just extracting BottleNeck Features, as we just want to learn generic features.

In [25]:
def forward_pass_Resnt():
        X_train, X_test, y_train, y_test = extract_features("./Data/")
        
        vgg_bottleneck = VGG16(weights='imagenet', include_top=False, pooling=POOLING)
        train_vgg_bf = vgg_bottleneck.predict(X_train, batch_size=32, verbose=1)
        valid_vgg_bf = vgg_bottleneck.predict(X_test, batch_size=32, verbose=1)
        
        xception_bottleneck = xception.Xception(weights='imagenet', include_top=False, pooling=POOLING)
        train_x_bf = xception_bottleneck.predict(X_train, batch_size=32, verbose=1)
        valid_x_bf = xception_bottleneck.predict(X_test, batch_size=32, verbose=1)
        
        X = np.hstack([train_x_bf, train_vgg_bf])
        V = np.hstack([valid_x_bf, valid_vgg_bf])
        
        logreg = LogisticRegression(multi_class='multinomial', solver='lbfgs', random_state=147)
        logreg.fit(X, y_train)
        valid_probs = logreg.predict_proba(V)
        valid_preds = logreg.predict(V)
        print ("validation probability: {}".format(valid_probs))
        print ("validation_predictions: {}".format(valid_preds))
        print ("Log Loss of Test Set:{}".format(log_loss(y_test, valid_probs)))
        print ("Accuracy:{}".format(accuracy_score(y_test, valid_preds)))

In [26]:
forward_pass_Resnt()

Instructions for updating:
Colocations handled automatically by placer.
562/562 [==============================] - 52s 92ms/step
validation probability: [[8.15260166e-10 4.05480411e-05 2.07724944e-06 ... 2.12100142e-06
  6.96000480e-07 1.67403397e-10]
 [5.61651084e-10 5.12826632e-04 2.41153202e-06 ... 1.52632636e-07
  8.68278717e-08 3.07491540e-07]
 [1.89211318e-06 6.53331336e-01 1.81652320e-09 ... 1.47400508e-09
  2.89315014e-04 3.98302790e-07]
 ...
 [5.38668248e-05 3.01452321e-05 1.54645021e-05 ... 1.08632960e-06
  3.57020128e-03 2.68380120e-05]
 [8.09151759e-11 3.79274751e-04 6.03157964e-08 ... 1.92158131e-11
  4.56226225e-09 2.70452098e-10]
 [8.91014809e-10 9.99999999e-01 6.26224869e-15 ... 2.34245954e-18
  6.41301279e-13 4.27816778e-22]]
validation_predictions: [ 6  4  1  1  1  3  1  1  1  1  2  3  4  1  5  1  4  1  3  3  1  1  5  2
  1  2  4  1  5  3  2  0  8  1  3  3  4 10  4  4  2  0  1  3  4  1  3  2
  5  4  8  5  1  3  7  1  4  2  3  0  4  1  4  5  1  0  2  1  4  4  5  7
  5 

/Users/sjadon/anaconda3/envs/mysite/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
